https://github.com/AMGitsKriss/GWI_Task

# Data Wrangling


**1. Describe the data: what values it mostly contains, what is the size of the dataset etc.**  
Prior to processing there are 100,200 entries in the dataset and more columns/features than are relevant. Most of these values appear to be one's and zeroes. Based on the task, the dataset is clearly intended to simulate the results of a multiple-choice questionnaire.

**2. Drop all duplicate respondents based on `respondent_index` column (keep the first occurrence of the respondent)**  
Easily doable with a dataframe. The functions `dataset.groupby(dataset.index).nth(0)` will deal with this. I originally tried to use the `.first()` function, but when I ran it on a smaller set of dummy data I found it was ignoring rows with NaN values. Since I'm being asked to take the first value, and not yet being expected to remove NaN values, `.nth(0)` is what I want.

**3. We will compute weighting based on the `q2` (gender) and `q4` (age group) column. Drop all respondents having NaNs in these columns. Do you know why this could happen and what the possible solutions are?**  
Dropping NaN values from specified columns is pretty straightforward: `dataset.dropna(subset=['q2', 'q4'])`. 

The NaN values would originate from users who have not specified their age or gender, presumably because they've *chosen* not to specify. In other instances it might stem from the user not even being presented with the question in particular. A common example would be "What country are you from?" If the user selects "US" there might be a field asking what state, but that field doesn't appear to anyone outside the US.

Typically you would either drop the NaN values, or fill them in with an average value for that column/feature. 

**4. There is a `q3_column.csv` file in the directory. Download it and merge it on the index to the original dataframe. Are there more variants of (a database) merge?**  

*"Are there more variants of (a database) merge?"*  
I'm uncertain what's meant by this. The question could refer to actions such as merge, join and concatenate, or it could refer to alternatives that come from the database type. For instance, in a relational database you wouldn't need to merge entire tables together; rather you'd just have entires that point to one another.

With regard to merging the two tables, I loaded the `q3_column.csv` in as a Series and gave it the name "q3". Then I used the Pandas concat function like so: `df.concat([df, q3], axis=1)`

# Computation

The factor is obviously what we're using to weight by. Weighting results like this is "target-size over actual-size," and both `quote_size` and `count` are unused. Thus we do `factor = quota_size/count`, which works. Simple.

# Bonus Task 

"... then write a Python script that reads this file line by line and writes data into teh database"

Assuming this to mean that each row needs pushing in turn, rather than uploading (or importing) the entire file into the database directly.

**Elasticsearch**

I installed the elasticsearch instance and left most settings on default, with the exception of the RAM allocation, which at a default of 2GB, is more than an AWM t2.micro instance has available. The index also had to be configured to accept more fields than the default 1000.

**The Script**

The script loads in the csv file as a Pandas dataframe, we declare an Elasticsearch connection object and use it to push each row to the database as we process the file. 

Iterating over the dataframe, we can hand each respondant index and the series that represents their data to a function that builds a dictionary that Elasticsearch will accept as an entry. We return it and push it to the server. This appears to take time to run, so this is where I stopped. 